In [2]:
from datasets import load_dataset
from tqdm import tqdm
import re
import os
import json

accept = {'melayu', 'malaysia', 'malaysian', 'malay'}

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    return set(string.split())

def exist(string):
    return len(clean(string) & accept) > 0

In [2]:
dataset = load_dataset("EleutherAI/the_pile_deduplicated", 'en', split='train', streaming = True)

Resolving data files:   0%|          | 0/1650 [00:00<?, ?it/s]

In [4]:
# !rm -rf c4
!mkdir c4

In [7]:
index = 0
for d in tqdm(dataset):
    if exist(d['text']):
        filename = os.path.join('c4', f'{index}.json')
        with open(filename, 'w') as fopen:
            json.dump(d['text'], fopen)
        index += 1

121203234it [21:17:03, 2100.09it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

122492188it [21:31:23, 2227.13it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

123824617it [21:45:58, 1667.35it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit

In [1]:
from glob import glob

files = glob('c4/*.json')

In [3]:
with open('c4-filtered.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
        fopen_l.write(f'{json.dumps(data)}\n')

100%|██████████| 508685/508685 [23:00<00:00, 368.39it/s]  


In [4]:
!ls -lh c4-filtered.jsonl

-rw-r--r-- 1 jovyan users 28G Nov 21 02:02 c4-filtered.jsonl


In [5]:
from huggingface_hub import HfApi

api = HfApi()

In [ ]:
api.upload_file(
    path_or_fileobj='c4-filtered.jsonl',
    path_in_repo="c4-filtered.jsonl",
    repo_id='malaysia-ai/dedup-text-dataset',
    repo_type="dataset",
)